In [1]:
import boto3
import json
import hashlib
import psycopg2
from hashlib import md5

In [2]:
# AWS SQS configuration
queue_url = 'http://localhost:4566/000000000000/login-queue'
sqs = boto3.client('sqs', region_name='us-east-2', endpoint_url=queue_url,
                   aws_access_key_id='dummy', aws_secret_access_key='dummy')

In [3]:
# Postgres configuration
conn = psycopg2.connect(
    dbname='postgres',
    user='postgres', 
    password='postgres',
    host='localhost', 
    port='5432'
)
cur = conn.cursor()


In [4]:
def main():
    # Receive messages from the SQS queue
    response = sqs.receive_message(
        QueueUrl=queue_url,
        MaxNumberOfMessages=10,
        VisibilityTimeout=0,
        WaitTimeSeconds=0
    )    

    if 'Messages' in response:
        for message in response['Messages']:
            message = response['Messages'][0]
            body = json.loads(message['Body'])
            print(message)

            # Hash the device_id and ip for masking and duplicate identification
            device_id = body.get('device_id', '19991')
            ip = body.get('ip', '0.0.0.0')
            masked_device_id = hashlib.md5(device_id.encode()).hexdigest()
            masked_ip = hashlib.md5(ip.encode()).hexdigest()
            print(masked_device_id)
            print(masked_ip)

            # Insert the masked data into user_logins table
            with conn:
                with conn.cursor() as cur:
                    insert_query = """
                        INSERT INTO user_logins (user_id, device_type, masked_ip, masked_device_id, locale, app_version, create_date)
                        VALUES (%s, %s, %s, %s, %s, %s, NOW())
                    """
                    cur.execute(insert_query, (
                        body['user_id'],
                        body['device_type'],
                        masked_ip,
                        masked_device_id,
                        body['locale'],
                        int(body['app_version'].replace('.', '')) if body['app_version'] else None
                    ))
                    conn.commit()

            # Delete the message from the queue after processing
            sqs.delete_message(
                QueueUrl=queue_url,
                ReceiptHandle=message['ReceiptHandle']
            )

    else:
        print('No messages in the queue')

if __name__ == "__main__":
    main()


{'MessageId': '957d4b3e-ddea-4928-b444-f32f994929e3', 'ReceiptHandle': 'NGIyMGU3ODgtYTA5Yi00MDg0LWEyYzQtNjQ2MzA1ZDZjMTEyIGFybjphd3M6c3FzOnVzLWVhc3QtMTowMDAwMDAwMDAwMDA6bG9naW4tcXVldWUgOTU3ZDRiM2UtZGRlYS00OTI4LWI0NDQtZjMyZjk5NDkyOWUzIDE2OTIzMTYzNTQuNzM1NTYzNQ==', 'MD5OfBody': '32365e026658d33521484837856cc808', 'Body': '{"foo": "oops_wrong_msg_type", "bar": "123"}'}
46bcb8f7477c1c33cf6dd9dd5c8045a3
f1f17934834ae2613699701054ef9684


KeyError: 'user_id'